In [3]:
import torch
import cv2
import numpy as np
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from torchvision import transforms

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN()

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Load the webcam
cap = cv2.VideoCapture(0)

# Image preprocessing transform
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255])  # Normalize to [-1, 1]
])

# Create directory for storing embeddings if it doesn't exist
embeddings_dir = 'face_embeddings'
if not os.path.exists(embeddings_dir):
    os.makedirs(embeddings_dir)

# Load existing face embeddings
def load_embeddings():
    embeddings = {}
    if os.path.exists(embeddings_dir):
        for filename in os.listdir(embeddings_dir):
            if filename.endswith('.pth'):
                name = os.path.splitext(filename)[0]
                path = os.path.join(embeddings_dir, filename)
                embeddings[name] = torch.load(path)
    return embeddings

# Function to check if face already exists
def face_exists(new_embedding, saved_embeddings, threshold=0.7):
    if not saved_embeddings:
        return False, None
    
    for name, embedding in saved_embeddings.items():
        # Calculate cosine similarity
        similarity = torch.nn.functional.cosine_similarity(new_embedding, embedding)
        if similarity > threshold:
            return True, name
    
    return False, None

# Function to save the face embedding
def save_face_embedding(face_tensor, name, saved_embeddings):
    # Get the face embedding
    with torch.no_grad():
        embedding = model(face_tensor).detach()
    
    # Check if face already exists
    exists, existing_name = face_exists(embedding, saved_embeddings)
    
    if exists:
        print(f"Face already exists as '{existing_name}'")
        return False
    
    # Save the embedding to a file (in .pth format)
    save_path = os.path.join(embeddings_dir, f"{name}.pth")
    torch.save(embedding, save_path)
    print(f"Face embedding for '{name}' saved successfully!")
    return True

# Load existing embeddings
saved_embeddings = load_embeddings()
print(f"Loaded {len(saved_embeddings)} existing face embeddings")

# State variables
save_mode = False
input_name = ""
current_face_tensor = None

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Create a copy of the frame for display
    display_frame = frame.copy()
    
    # Detect faces
    faces, _ = mtcnn.detect(frame)
    
    if faces is not None:
        for face in faces:
            if face is not None:
                # Draw bounding box around the detected face
                x1, y1, x2, y2 = [int(coord) for coord in face]
                cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Crop the face from the frame
                face_crop = frame[y1:y2, x1:x2]
                
                if face_crop.size != 0:
                    try:
                        # Preprocess the cropped face
                        face_tensor = preprocess(face_crop)
                        
                        # Add batch dimension (for model compatibility)
                        face_tensor = face_tensor.unsqueeze(0)
                        
                        current_face_tensor = face_tensor
                        
                        # If not in save mode, display instructions
                        if not save_mode:
                            cv2.putText(display_frame, 'Press "s" to save this face', 
                                        (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    except Exception as e:
                        print(f"Error processing face: {e}")
    
    # If in save mode, display the text input interface
    if save_mode:
        cv2.putText(display_frame, f'Enter name: {input_name}_', 
                    (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(display_frame, 'Press ENTER to confirm or ESC to cancel', 
                    (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
    
    # Show the frame
    cv2.imshow('Face Recognition - Save Face', display_frame)

    # Key handling
    key = cv2.waitKey(1) & 0xFF
    
    if save_mode:
        # Handle text input
        if key == 13:  # Enter key
            if input_name and current_face_tensor is not None:
                success = save_face_embedding(current_face_tensor, input_name, saved_embeddings)
                if success:
                    saved_embeddings = load_embeddings()  # Reload embeddings
                save_mode = False
                input_name = ""
        elif key == 27:  # Esc key
            save_mode = False
            input_name = ""
        elif key == 8:  # Backspace
            input_name = input_name[:-1] if input_name else ""
        elif key >= 32 and key <= 126:  # Printable ASCII characters
            input_name += chr(key)
    else:
        # Normal mode key handling
        if key == ord('s') and current_face_tensor is not None:
            save_mode = True
        elif key == 27:  # Esc key to exit
            break

cap.release()
cv2.destroyAllWindows()

Loaded 0 existing face embeddings
Face embedding for 'Gaindu' saved successfully!


In [ ]:
import torch
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine
from torchvision import transforms

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN()

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Load the saved face embedding (if it exists)
saved_embedding = torch.load('my_face_embeddings.pth')

# Load the webcam
cap = cv2.VideoCapture(0)

# Threshold for face recognition (cosine distance threshold)
threshold = 0.6

# Image preprocessing pipeline
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255])  # Normalize to [-1, 1]
])

# Function to recognize face by comparing embeddings
def recognize_face(face_tensor):
    # Get the face embedding
    embedding = model(face_tensor)
    
    # Flatten both the saved embedding and the current face embedding to 1D
    embedding_flat = embedding.flatten()
    saved_embedding_flat = saved_embedding.flatten()
    
    # Calculate the cosine distance between saved embedding and the current face embedding
    distance = cosine(saved_embedding_flat.detach().numpy(), embedding_flat.detach().numpy())
    print(f'Cosine distance: {distance}')
    
    if distance < threshold:
        print("User recognized: Gaindu")  # Print the recognized user on the console
        return "Gaindu"
    else:
        print("User recognized: Unknown")  # Print "Unknown" on the console if not recognized
        return "Unknown"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    faces, _ = mtcnn.detect(frame)
    
    if faces is not None:
        for face in faces:
            # Draw bounding box around the detected face
            cv2.rectangle(frame, 
                          (int(face[0]), int(face[1])), 
                          (int(face[2]), int(face[3])), 
                          (0, 255, 0), 2)
            
            # Crop the face from the frame
            face_crop = frame[int(face[1]):int(face[3]), int(face[0]):int(face[2])]
            
            if face_crop.size != 0:
                # Preprocess the cropped face
                face_tensor = preprocess(face_crop)

                # Add batch dimension (for model compatibility)
                face_tensor = face_tensor.unsqueeze(0)

                # Recognize the face
                label = recognize_face(face_tensor)

                # Display the label on the image
                cv2.putText(frame, label, (int(face[0]), int(face[1]) - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Show the frame with bounding boxes and labels
        cv2.imshow('Face Recognition', frame)

    # Exit when the 'Esc' key is pressed (key code 27)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch
import cv2
import numpy as np
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from scipy.spatial.distance import cosine
from torchvision import transforms

# Initialize MTCNN (Face Detection)
mtcnn = MTCNN()

# Initialize Inception Resnet (Face Recognition)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Directory where face embeddings are stored
embeddings_dir = 'face_embeddings'

# Load the webcam
cap = cv2.VideoCapture(0)

# Threshold for face recognition (cosine distance threshold)
threshold = 0.6

# Image preprocessing pipeline
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0, 0, 0], std=[255, 255, 255])  # Normalize to [-1, 1]
])

# Function to load all saved face embeddings
def load_embeddings():
    embeddings = {}
    if os.path.exists(embeddings_dir):
        for filename in os.listdir(embeddings_dir):
            if filename.endswith('.pth'):
                name = os.path.splitext(filename)[0]
                path = os.path.join(embeddings_dir, filename)
                embeddings[name] = torch.load(path)
    return embeddings

# Load saved embeddings
saved_embeddings = load_embeddings()
print(f"Loaded {len(saved_embeddings)} face embeddings")

# Function to recognize face
def recognize_face(face_tensor):
    # Get the face embedding
    with torch.no_grad():
        embedding = model(face_tensor).detach()
    
    # Find the closest match
    min_distance = float('inf')
    recognized_name = "Unknown"
    
    for name, saved_embedding in saved_embeddings.items():
        # Calculate cosine distance
        distance = cosine(embedding.flatten().detach().numpy(), 
                         saved_embedding.flatten().detach().numpy())
        
        print(f"Distance from {name}: {distance:.4f}")
        
        if distance < min_distance:
            min_distance = distance
            recognized_name = name
    
    # Check if the minimum distance is below our threshold
    if min_distance < threshold:
        print(f"User recognized: {recognized_name} (distance: {min_distance:.4f})")
        return recognized_name
    else:
        print(f"User unknown (min distance: {min_distance:.4f})")
        return "Unknown"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces
    faces, _ = mtcnn.detect(frame)
    
    if faces is not None and faces.shape[0] > 0:
        for face in faces:
            if face is not None:
                # Draw bounding box around the detected face
                x1, y1, x2, y2 = [int(coord) for coord in face]
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Crop the face from the frame
                face_crop = frame[y1:y2, x1:x2]
                
                if face_crop.size != 0:
                    try:
                        # Preprocess the cropped face
                        face_tensor = preprocess(face_crop)
                        
                        # Add batch dimension (for model compatibility)
                        face_tensor = face_tensor.unsqueeze(0)
                        
                        # Recognize the face
                        label = recognize_face(face_tensor)
                        
                        # Display the label on the image
                        cv2.putText(frame, label, (x1, y1 - 10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)
                    except Exception as e:
                        print(f"Error processing face: {e}")

    # Show the frame with bounding boxes and labels
    cv2.imshow('Face Recognition', frame)

    # Exit when the 'Esc' key is pressed (key code 27)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()